# ***Step 1: Import libraries & Set up***

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
sequence_length = 28
input_size = 12 # N_mffc
hidden_size = 64
num_layers = 3
num_classes = 35
batch_size = 1
num_epochs = 25
learning_rate = 0.001

# ***Step 2: Use trained model (FP32)***
Reference: https://github.com/felixchenfy/Speech-Commands-Classification-by-LSTM-PyTorch

In [ ]:
# Recurrent neural network (many-to-one)
class LSTM(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        return x
model = LSTM(12, 35,64,3)
x = torch.randn(batch_size, sequence_length, input_size)
out = model(x)
print(out)



tensor([[[ 0.3469, -1.4874, -0.1450, -0.6015,  0.4447, -0.3941, -0.1339,
          -1.1706,  0.3192,  0.1349,  0.4416, -0.0699],
         [-0.0342, -0.1513,  1.1628,  0.3381, -2.4266,  0.3415,  1.1566,
          -1.7085,  0.9702, -0.4843,  1.2344,  0.4435],
         [ 1.6728, -1.0947,  1.5710,  1.7005,  0.9928, -0.6565, -2.0365,
           0.0337, -0.6766, -0.6277, -0.4878,  0.2219],
         [ 0.6002, -0.6824, -0.6465, -0.9033,  1.2781, -0.4886,  0.2029,
           1.1325,  0.1463,  1.5354,  1.1932,  0.0048],
         [-0.2679, -1.1787, -1.6024,  0.3694, -0.9934,  0.5546, -1.2586,
           0.5416, -0.2124,  1.5033, -0.4508, -1.9895],
         [ 2.5777,  1.7366, -0.9346,  0.2837, -0.8152, -0.3704,  0.2418,
           0.0614, -0.3858,  1.4835, -0.0736,  0.4397],
         [-0.9950,  0.6258,  0.1800,  2.7300, -1.5928,  0.2763, -0.1658,
          -0.4066,  0.4823, -1.3553,  1.1546,  0.8119],
         [ 0.8891,  0.1830,  0.3364,  0.3167, -1.1692,  0.3125, -0.2180,
          -0.8463,  0.78

***--- Above LSTM model tensors representation for Affine per tensor quantization--***

In [ ]:
import os

In [ ]:
T = torch.tensor(out)
print(T)

tensor([[[ 0.3469, -1.4874, -0.1450, -0.6015,  0.4447, -0.3941, -0.1339,
          -1.1706,  0.3192,  0.1349,  0.4416, -0.0699],
         [-0.0342, -0.1513,  1.1628,  0.3381, -2.4266,  0.3415,  1.1566,
          -1.7085,  0.9702, -0.4843,  1.2344,  0.4435],
         [ 1.6728, -1.0947,  1.5710,  1.7005,  0.9928, -0.6565, -2.0365,
           0.0337, -0.6766, -0.6277, -0.4878,  0.2219],
         [ 0.6002, -0.6824, -0.6465, -0.9033,  1.2781, -0.4886,  0.2029,
           1.1325,  0.1463,  1.5354,  1.1932,  0.0048],
         [-0.2679, -1.1787, -1.6024,  0.3694, -0.9934,  0.5546, -1.2586,
           0.5416, -0.2124,  1.5033, -0.4508, -1.9895],
         [ 2.5777,  1.7366, -0.9346,  0.2837, -0.8152, -0.3704,  0.2418,
           0.0614, -0.3858,  1.4835, -0.0736,  0.4397],
         [-0.9950,  0.6258,  0.1800,  2.7300, -1.5928,  0.2763, -0.1658,
          -0.4066,  0.4823, -1.3553,  1.1546,  0.8119],
         [ 0.8891,  0.1830,  0.3364,  0.3167, -1.1692,  0.3125, -0.2180,
          -0.8463,  0.78

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


***--Maximum Value and minimum value of x***

In [ ]:
# Min-Max value of float_32 tensor (x) find out for scale (s) and zero point (z)

b = torch.max(T)
print(b)

tensor(2.7300)


In [ ]:
a = torch.min(T)
print(a)

tensor(-3.3744)


In [ ]:
# scale value

s = (b-a)/255

print(s)

tensor(0.0239)


In [ ]:
# zero point

z = torch.round(-a*255/(b-a))

print(z)

tensor(141.)


# ***Step-3: Apply Quantization by round and clipping function (Affine mapping)***

In [ ]:
f = torch.round(T/s + z)
print(f)
Tq = torch.clip(f, min=0, max=255) # Here min & max value we can change as per Tbit.
# But, I have checked for 8 bit
print (Tq)
torch.save(Tq,'qtz_tensor.pt')


tensor([[[155.,  79., 135., 116., 160., 125., 135.,  92., 154., 147., 159.,
          138.],
         [140., 135., 190., 155.,  40., 155., 189.,  70., 182., 121., 193.,
          160.],
         [211.,  95., 207., 212., 182., 114.,  56., 142., 113., 115., 121.,
          150.],
         [166., 112., 114., 103., 194., 121., 149., 188., 147., 205., 191.,
          141.],
         [130.,  92.,  74., 156., 100., 164.,  88., 164., 132., 204., 122.,
           58.],
         [249., 214., 102., 153., 107., 126., 151., 144., 125., 203., 138.,
          159.],
         [ 99., 167., 149., 255.,  74., 153., 134., 124., 161.,  84., 189.,
          175.],
         [178., 149., 155., 154.,  92., 154., 132., 106., 174., 155., 151.,
          182.],
         [200., 114., 158., 108., 162., 101., 141., 188., 107., 185., 172.,
          128.],
         [111.,  72., 115., 137., 140., 142., 120., 107., 221., 159., 133.,
          154.],
         [ 51., 164., 117., 127., 129., 156., 152., 161., 149., 147., 

# ***Step-4: Apply Dequantization***

In [ ]:
Tdq = s*(Tq - z)
print(Tdq)

tensor([[[ 0.3351, -1.4842, -0.1436, -0.5985,  0.4548, -0.3830, -0.1436,
          -1.1730,  0.3112,  0.1436,  0.4309, -0.0718],
         [-0.0239, -0.1436,  1.1730,  0.3351, -2.4178,  0.3351,  1.1491,
          -1.6997,  0.9815, -0.4788,  1.2448,  0.4548],
         [ 1.6757, -1.1012,  1.5800,  1.6997,  0.9815, -0.6464, -2.0348,
           0.0239, -0.6703, -0.6224, -0.4788,  0.2155],
         [ 0.5985, -0.6942, -0.6464, -0.9097,  1.2688, -0.4788,  0.1915,
           1.1251,  0.1436,  1.5321,  1.1969,  0.0000],
         [-0.2633, -1.1730, -1.6039,  0.3591, -0.9815,  0.5506, -1.2688,
           0.5506, -0.2155,  1.5082, -0.4548, -1.9869],
         [ 2.5854,  1.7475, -0.9336,  0.2873, -0.8139, -0.3591,  0.2394,
           0.0718, -0.3830,  1.4842, -0.0718,  0.4309],
         [-1.0054,  0.6224,  0.1915,  2.7290, -1.6039,  0.2873, -0.1676,
          -0.4070,  0.4788, -1.3645,  1.1491,  0.8139],
         [ 0.8857,  0.1915,  0.3351,  0.3112, -1.1730,  0.3112, -0.2155,
          -0.8379,  0.79

# ***Step-5: MAE/MSE loss between T and Tdq***

***I. MAE loss***

In [ ]:
# Import the required libraries
import torch
import torch.nn as nn
# print input and target tensors
print("Input Tensor:\n", T)
print("Target Tensor:\n", Tdq)
# create a criterion to measure the mean absolute error
mae = nn.L1Loss()
# compute the loss (mean absolute error)
output = mae(T, Tdq)
# output.backward()
print("MAE loss:", output)

Input Tensor:
 tensor([[[ 0.3469, -1.4874, -0.1450, -0.6015,  0.4447, -0.3941, -0.1339,
          -1.1706,  0.3192,  0.1349,  0.4416, -0.0699],
         [-0.0342, -0.1513,  1.1628,  0.3381, -2.4266,  0.3415,  1.1566,
          -1.7085,  0.9702, -0.4843,  1.2344,  0.4435],
         [ 1.6728, -1.0947,  1.5710,  1.7005,  0.9928, -0.6565, -2.0365,
           0.0337, -0.6766, -0.6277, -0.4878,  0.2219],
         [ 0.6002, -0.6824, -0.6465, -0.9033,  1.2781, -0.4886,  0.2029,
           1.1325,  0.1463,  1.5354,  1.1932,  0.0048],
         [-0.2679, -1.1787, -1.6024,  0.3694, -0.9934,  0.5546, -1.2586,
           0.5416, -0.2124,  1.5033, -0.4508, -1.9895],
         [ 2.5777,  1.7366, -0.9346,  0.2837, -0.8152, -0.3704,  0.2418,
           0.0614, -0.3858,  1.4835, -0.0736,  0.4397],
         [-0.9950,  0.6258,  0.1800,  2.7300, -1.5928,  0.2763, -0.1658,
          -0.4066,  0.4823, -1.3553,  1.1546,  0.8119],
         [ 0.8891,  0.1830,  0.3364,  0.3167, -1.1692,  0.3125, -0.2180,
         

***II. MSE Loss***

In [ ]:
# Import the required libraries
import torch
import torch.nn as nn

# print input and target tensors
print("Input Tensor:\n", T)
print("Target Tensor:\n", Tdq)

# create a criterion to measure the mean squared error
mse = nn.MSELoss()

# compute the loss (mean squared error)
output = mse(T, Tdq)

# output.backward()
print("MSE loss:", output)

Input Tensor:
 tensor([[[ 0.3469, -1.4874, -0.1450, -0.6015,  0.4447, -0.3941, -0.1339,
          -1.1706,  0.3192,  0.1349,  0.4416, -0.0699],
         [-0.0342, -0.1513,  1.1628,  0.3381, -2.4266,  0.3415,  1.1566,
          -1.7085,  0.9702, -0.4843,  1.2344,  0.4435],
         [ 1.6728, -1.0947,  1.5710,  1.7005,  0.9928, -0.6565, -2.0365,
           0.0337, -0.6766, -0.6277, -0.4878,  0.2219],
         [ 0.6002, -0.6824, -0.6465, -0.9033,  1.2781, -0.4886,  0.2029,
           1.1325,  0.1463,  1.5354,  1.1932,  0.0048],
         [-0.2679, -1.1787, -1.6024,  0.3694, -0.9934,  0.5546, -1.2586,
           0.5416, -0.2124,  1.5033, -0.4508, -1.9895],
         [ 2.5777,  1.7366, -0.9346,  0.2837, -0.8152, -0.3704,  0.2418,
           0.0614, -0.3858,  1.4835, -0.0736,  0.4397],
         [-0.9950,  0.6258,  0.1800,  2.7300, -1.5928,  0.2763, -0.1658,
          -0.4066,  0.4823, -1.3553,  1.1546,  0.8119],
         [ 0.8891,  0.1830,  0.3364,  0.3167, -1.1692,  0.3125, -0.2180,
         

# ***Method-2:- Symmetric quantization scheme***

In [ ]:
f = torch.round(T/s + z)
print(f)
Tq1 = torch.clip(f, min=-128, max=128) # Here min & max value we can change as per Tbit.
# But, I have checked for 8 bit
print (Tq1)
torch.save(Tq1,'qtz_tensor.pt')


tensor([[[155.,  79., 135., 116., 160., 125., 135.,  92., 154., 147., 159.,
          138.],
         [140., 135., 190., 155.,  40., 155., 189.,  70., 182., 121., 193.,
          160.],
         [211.,  95., 207., 212., 182., 114.,  56., 142., 113., 115., 121.,
          150.],
         [166., 112., 114., 103., 194., 121., 149., 188., 147., 205., 191.,
          141.],
         [130.,  92.,  74., 156., 100., 164.,  88., 164., 132., 204., 122.,
           58.],
         [249., 214., 102., 153., 107., 126., 151., 144., 125., 203., 138.,
          159.],
         [ 99., 167., 149., 255.,  74., 153., 134., 124., 161.,  84., 189.,
          175.],
         [178., 149., 155., 154.,  92., 154., 132., 106., 174., 155., 151.,
          182.],
         [200., 114., 158., 108., 162., 101., 141., 188., 107., 185., 172.,
          128.],
         [111.,  72., 115., 137., 140., 142., 120., 107., 221., 159., 133.,
          154.],
         [ 51., 164., 117., 127., 129., 156., 152., 161., 149., 147., 

In [ ]:
Tdq1 = s*(Tq1 - z)
print(Tdq1)

tensor([[[-0.3112, -1.4842, -0.3112, -0.5985, -0.3112, -0.3830, -0.3112,
          -1.1730, -0.3112, -0.3112, -0.3112, -0.3112],
         [-0.3112, -0.3112, -0.3112, -0.3112, -2.4178, -0.3112, -0.3112,
          -1.6997, -0.3112, -0.4788, -0.3112, -0.3112],
         [-0.3112, -1.1012, -0.3112, -0.3112, -0.3112, -0.6464, -2.0348,
          -0.3112, -0.6703, -0.6224, -0.4788, -0.3112],
         [-0.3112, -0.6942, -0.6464, -0.9097, -0.3112, -0.4788, -0.3112,
          -0.3112, -0.3112, -0.3112, -0.3112, -0.3112],
         [-0.3112, -1.1730, -1.6039, -0.3112, -0.9815, -0.3112, -1.2688,
          -0.3112, -0.3112, -0.3112, -0.4548, -1.9869],
         [-0.3112, -0.3112, -0.9336, -0.3112, -0.8139, -0.3591, -0.3112,
          -0.3112, -0.3830, -0.3112, -0.3112, -0.3112],
         [-1.0054, -0.3112, -0.3112, -0.3112, -1.6039, -0.3112, -0.3112,
          -0.4070, -0.3112, -1.3645, -0.3112, -0.3112],
         [-0.3112, -0.3112, -0.3112, -0.3112, -1.1730, -0.3112, -0.3112,
          -0.8379, -0.31

In [ ]:
# Import the required libraries
import torch
import torch.nn as nn
# print input and target tensors
print("Input Tensor:\n", T)
print("Target Tensor:\n", Tdq1)
# create a criterion to measure the mean absolute error
mae = nn.L1Loss()
# compute the loss (mean absolute error)
output = mae(T, Tdq1)
# output.backward()
print("MAE loss:", output)

Input Tensor:
 tensor([[[ 0.3469, -1.4874, -0.1450, -0.6015,  0.4447, -0.3941, -0.1339,
          -1.1706,  0.3192,  0.1349,  0.4416, -0.0699],
         [-0.0342, -0.1513,  1.1628,  0.3381, -2.4266,  0.3415,  1.1566,
          -1.7085,  0.9702, -0.4843,  1.2344,  0.4435],
         [ 1.6728, -1.0947,  1.5710,  1.7005,  0.9928, -0.6565, -2.0365,
           0.0337, -0.6766, -0.6277, -0.4878,  0.2219],
         [ 0.6002, -0.6824, -0.6465, -0.9033,  1.2781, -0.4886,  0.2029,
           1.1325,  0.1463,  1.5354,  1.1932,  0.0048],
         [-0.2679, -1.1787, -1.6024,  0.3694, -0.9934,  0.5546, -1.2586,
           0.5416, -0.2124,  1.5033, -0.4508, -1.9895],
         [ 2.5777,  1.7366, -0.9346,  0.2837, -0.8152, -0.3704,  0.2418,
           0.0614, -0.3858,  1.4835, -0.0736,  0.4397],
         [-0.9950,  0.6258,  0.1800,  2.7300, -1.5928,  0.2763, -0.1658,
          -0.4066,  0.4823, -1.3553,  1.1546,  0.8119],
         [ 0.8891,  0.1830,  0.3364,  0.3167, -1.1692,  0.3125, -0.2180,
         

In [ ]:
# Import the required libraries
import torch
import torch.nn as nn

# print input and target tensors
print("Input Tensor:\n", T)
print("Target Tensor:\n", Tdq1)

# create a criterion to measure the mean squared error
mse = nn.MSELoss()

# compute the loss (mean squared error)
output = mse(T, Tdq1)

# output.backward()
print("MSE loss:", output)

Input Tensor:
 tensor([[[ 0.3469, -1.4874, -0.1450, -0.6015,  0.4447, -0.3941, -0.1339,
          -1.1706,  0.3192,  0.1349,  0.4416, -0.0699],
         [-0.0342, -0.1513,  1.1628,  0.3381, -2.4266,  0.3415,  1.1566,
          -1.7085,  0.9702, -0.4843,  1.2344,  0.4435],
         [ 1.6728, -1.0947,  1.5710,  1.7005,  0.9928, -0.6565, -2.0365,
           0.0337, -0.6766, -0.6277, -0.4878,  0.2219],
         [ 0.6002, -0.6824, -0.6465, -0.9033,  1.2781, -0.4886,  0.2029,
           1.1325,  0.1463,  1.5354,  1.1932,  0.0048],
         [-0.2679, -1.1787, -1.6024,  0.3694, -0.9934,  0.5546, -1.2586,
           0.5416, -0.2124,  1.5033, -0.4508, -1.9895],
         [ 2.5777,  1.7366, -0.9346,  0.2837, -0.8152, -0.3704,  0.2418,
           0.0614, -0.3858,  1.4835, -0.0736,  0.4397],
         [-0.9950,  0.6258,  0.1800,  2.7300, -1.5928,  0.2763, -0.1658,
          -0.4066,  0.4823, -1.3553,  1.1546,  0.8119],
         [ 0.8891,  0.1830,  0.3364,  0.3167, -1.1692,  0.3125, -0.2180,
         